# Hierarchický index (pokročilí)

Pokročilejší práce s multiindexem (hierarchický index).

* Vyřešíme soustavu rovnic. Naučíme se uložit data do tabulky, kde budou přístupné výsledky pro jednotlivá nastavení modelu (datasety) a pro vývoj jednotlivých populací. 
* Naučíme se uložená data vyvolávat a používat například pro tvorbu grafů.

Nejprve import knihoven a nastavení.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp

def konkurence(t,X,a=1,b=1,c=0.4,alpha=1,beta=.2,gamma=1):
    x,y = X
    return [(a-b*x-c*y)*x, (alpha-beta*x-gamma*y)*y]

meze = [0,30]
pocatecni_podminka = [0.2,0.3]
t = np.linspace(*meze,500)

## Tvorba tabulky s hierarchickým indexem

Vytvoříme tabulku se sloupcovým indexem obsahujícím dvě úrovně. Pro začátek uložíme pouze sloupec s časem (v tomto případě použijeme pouze jednu úroveň). Ideální je vytvořit sloupce současně s tabulkou a poté do nich jenom vkládat data (neprázdný seznam v parametru, ale je možné sloupce vytvářet i později.

In [ ]:
my_index = pd.MultiIndex.from_tuples([], names=['dataset', 'populace'])
df = pd.DataFrame(columns=my_index)
df["t"] = t
df.tail()

Dále vyřešíme pro jednotlivá nastavení (bez konkurence, s konkurencí a dominance jednoho a druhého druhu) a vložíme do tabulky. Vytiskneme konec tabulky.

In [ ]:
datasets = ["nezávislé populace","slabá konkurence","dominance x","dominance y"]  # Názvy datasetů
druhy = ["x","y"] # Názvy druhů

for c,beta,d in zip( 
                [0,0.4,0.4,1.5],   # hodnoty pro koeficient c 
                [0,0.2,1.3,0.8],   # hodnoty pro koeficient beta
                datasets  # názvy datasetů
        ):
    reseni = solve_ivp(
                       lambda t,X:konkurence(t,X,c=c,beta=beta),
                       meze,
                       pocatecni_podminka,
                       t_eval=t 
                       ).y.T   # řešení ve sloupcích
    df[[(d,druh) for druh in druhy]]=reseni # uložení do tabulky
df.tail() # tisk konce tabulky pro kontrolu

## Použití dat z tabulky

Na řadě je vykreslení dat ze sloupců tak, aby byly v jednom obrázku všechna data pro jednotlivé datasety a poté v jedno obrázku všechna data pro jednotlivé populace.

In [ ]:
fig,axs = plt.subplots(2,2,sharex=True, sharey=True)  # obrázek se čtyřmi podobrázky ve dou řadách a sloupcích
ax = axs.ravel()  # abychom se nemuseli na podobrázky odkazovat dvojitým indexem, seřadíme je lineárně
for a,dataset in zip(ax,datasets):  # cyklus přes podobrázky a datasety
    a.plot(t,df.xs(key=(dataset,"x"),level=(0,1),axis=1))  # vykreslení datasetu do podobrázku
    a.plot(t,df.xs(key=(dataset,"y"),level=(0,1),axis=1))  # vykreslení datasetu do podobrázku
    a.set(  # nastavení nadpisu
        title=dataset.capitalize()
        )
    a.grid()
ax[0].set(ylim=(0,1.05))    # rozsah pro svislou osu
ax[0].legend([f"Druh {i}" for i in druhy])  # legenda
plt.tight_layout()  # automatické upravení mezer mezi obrázky

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True)
for a,populace in zip(ax,druhy):
    a.plot(df["t"],df.xs(level=1,axis=1,key=populace))
    a.set(
        ylim=(0,1.05),
        title=f"Populace ${populace}$",
    )
ax[0].legend(datasets)
plt.tight_layout()

Mírnou modifikací kódu je možné vykreslit trajektorie řešení. (Pozor na to, jedná se o trajektorie pro čtyři různé autonomní systémy, proto trajektorie mohou vycházet z jednoho bodu por různými směry.)

In [ ]:
fig,ax = plt.subplots()
for dataset in datasets:
    ax.plot(df[dataset,"x"],df[dataset,"y"],label=dataset)
ax.legend()
ax.set(title="Trajektorie v modelu konkurence dvou populací")
plt.tight_layout()

## Ukázky selekce sloupců

Výběr podle hodnoty první úrově indexu je jednoduchý. Vlastně to je stejné jako pro klasické indexy. 

Nejprve jak vypadá naše tabulka (začátek).

In [ ]:
df.head()

Následovně se z dat vyfiltrje vývoj systému, kdy sledujeme dvě nezávislé populace.

In [ ]:
df["nezávislé populace"]

Komplikovanější je, když na hodnotě první úrovně indexu nezáleží, ale vybíráme podle druhé úrovně. Možností je několik, jedna z nich je využití příkazu `xs`. Takto se vyberou sloupce s vývojem první populace pro jednotlivé scénáře (datasety).

In [ ]:
df.xs(level=1,axis=1,key="x")

Pokud vybíráme podle více kritérií, používáme n-tice.

In [ ]:
df.xs(level=(0,1),axis=1,key=("slabá konkurence","x"))

Pokud chceme jeden konkrétní sloupec, je možné předchozí kód zjednodušit. Takto se vyberou data odpovídající první populaci při slabé konkurenci. Formálně je výsledek stejný jako v minulém případě, ale protože výsledekem je jeden sloupec, je výsledek ve tvaru vektoru (přesněji řada typu `pandas.Series`). Předchozí postup pomocí `xs` vrací tabulku, která by při existenci další hierarchie ve sloupcovém indexu mohla mít více sloupců

In [ ]:
df["slabá konkurence","x"]
